# GEE MODIS Download Scripts
Note: This notebook **must be run on Google Colab**. It uses Colab's integration with Google Earth Engine (GEE) to access MODIS data.

This script takes a long time to run. It may need to be run over several days, as the number of downloaded files is fairly large (even though they are individually small). Files will be aggregated and stored in Google Drive where they can be downloaded and added to the CarbonSense raw data directory.

In [ ]:
import os
import requests
import pickle as pkl
import concurrent
import json
import io

!pip install geojson
!pip install pyproj
!pip install rasterio
import numpy as np
import geopandas as gpd
import geojson
from pyproj import Proj, transform
from rasterio.io import MemoryFile
from rasterio import logging

import ee
import google
from google.api_core import retry
from google.colab import auth, drive
from tqdm.notebook import tqdm
log = logging.getLogger()
log.setLevel(logging.ERROR)

In [ ]:
PROJECT_NAME = '' # ex 'ee-username'

drive.mount('/content/gdrive')
auth.authenticate_user()
credentials, _ = google.auth.default()
ee.Initialize(credentials, project=PROJECT_NAME, opt_url='https://earthengine-highvolume.googleapis.com')

with open('/content/gdrive/MyDrive/sites.geojson', 'r') as f:
  sites = json.load(f)['features']

In [ ]:
def MCD43A4_image_dict():
  bands = [
    'Nadir_Reflectance_Band1',
    'Nadir_Reflectance_Band2',
    'Nadir_Reflectance_Band3',
    'Nadir_Reflectance_Band4',
    'Nadir_Reflectance_Band5',
    'Nadir_Reflectance_Band6',
    'Nadir_Reflectance_Band7',
  ]
  MCD43A4 = ee.ImageCollection('MODIS/061/MCD43A4').select(bands)
  image_dict = {}
  for i in range(2000, 2024):
    year_images = MCD43A4.filter(
        ee.Filter.calendarRange(i, i, 'year')
    ).getInfo()['features']
    for image in year_images:
      date = image['id'].split('/')[-1]
      image_dict[date] = image

  return { 'images': image_dict, 'bands': bands, 'product': 'MCD43A4.061' }


def MCD43A2_image_dict():
  bands = [
    'Snow_BRDF_Albedo',
    'BRDF_Albedo_Platform',
    'BRDF_Albedo_LandWaterType',
    'BRDF_Albedo_Band_Quality_Band1',
    'BRDF_Albedo_Band_Quality_Band2',
    'BRDF_Albedo_Band_Quality_Band3',
    'BRDF_Albedo_Band_Quality_Band4',
    'BRDF_Albedo_Band_Quality_Band5',
    'BRDF_Albedo_Band_Quality_Band6',
    'BRDF_Albedo_Band_Quality_Band7',
  ]
  MCD43A2 = ee.ImageCollection('MODIS/061/MCD43A2').select(bands)
  image_dict = {}
  for i in range(2000, 2024):
    year_images = MCD43A2.filter(
        ee.Filter.calendarRange(i, i, 'year')
    ).getInfo()['features']
    for image in year_images:
      date = image['id'].split('/')[-1]
      image_dict[date] = image

  return { 'images': image_dict, 'bands': bands, 'product': 'MCD43A2.061' }


def Sentinel_2_image_dict():
  bands = [
      'B1',
      'B2',
      'B3',
      'B4',
      'B5',
      'B6',
      'B7',
      'B8',
      'B8A',
      'B9',
      'B11',
      'B12'
  ]
  S2 = ee.ImageCollection('COPERNICUS/S2_SR').select(bands)
  image_dict = {}
  for i in range(2000, 2001):
    year_images = S2.filter(
        ee.Filter.calendarRange(i, i, 'year')
    ).getInfo()['features']
    for image in year_images:
      date = image['id'].split('/')[-1]
      image_dict[date] = image

  return { 'images': image_dict, 'bands': bands, 'product': 'COPERNICUS/S2' }



In [ ]:
def get_image_ids_for_site(site, image_dict, existing=[]):
  time_info = site['properties']['TIME_INFO']
  image_ids = []
  for _, v in time_info.items():
    image_ids.extend([image_dict['images'][d]['id'] for d in image_dict['images'].keys() if d >= v[0] and d <= v[1] and d not in existing])
  return image_ids


@retry.Retry()
def get_patch(geometry, asset_id, bands, grid=None, return_meta=False):
  """Get a patch of pixels from an asset, centered on the coords."""
  request = {
    'fileFormat': 'GEO_TIFF',
    'bandIds': bands,
    'region': geometry,
    'assetId': asset_id
  }
  if grid is not None:
    request['grid'] = grid
  resp = ee.data.getPixels(request)

  with MemoryFile(resp) as memfile:
    with memfile.open() as src:
      pixels = src.read()
      meta = src.meta

  if return_meta:
    return meta
  else:
    return pixels

In [ ]:
def latlon_to_utm_epsg(latitude, longitude):
    # Find UTM zone based on longitude
    utm_zone = int((longitude + 180) / 6) + 1
    # Determine hemisphere (northern or southern) to select the correct EPSG code
    if latitude >= 0:
        epsg_code = 32600 + utm_zone
    else:
        epsg_code = 32700 + utm_zone

    # Return the EPSG code along with UTM zone, easting, and northing
    return f'EPSG:{epsg_code}'


def get_site_data(site, image_dict, subdir, reproject=False):
  id = site['properties']['SITE_ID']
  grid = None
  if reproject:
    crs_code = latlon_to_utm_epsg(site['properties']['LOCATION_LAT'], site['properties']['LOCATION_LON'])
    grid = { 'crsCode': crs_code }

  # Check for existing records - no need to repull. This supports half-finished sites too.
  keys = []
  if os.path.isfile(f'/content/gdrive/MyDrive/{subdir}/{id}.pkl'):
    with open(f'/content/gdrive/MyDrive/{subdir}/{id}.pkl', 'rb') as f:
      data = pkl.load(f)
      keys = list(data['pixel_values'].keys())
      print(f'  Found existing file with {len(keys)} images')

  image_ids = get_image_ids_for_site(site, image_dict, existing=keys)

  if len(image_ids) == 0:
    print(f'  No satellite records found for {id}')
    return True
  else:
    print(f'  Pulling {len(image_ids)} images')
  geo = site['geometry']
  executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
  future_to_image = {
    executor.submit(get_patch, geo, id, image_dict['bands'], grid=grid): id for id in image_ids
  }

  pixel_values = {}
  for future in concurrent.futures.as_completed(future_to_image):
    image_id = future_to_image[future]
    image_date = image_id.split('/')[-1]
    try:
      np_array = future.result()
      pixel_values[image_date] = np_array
    except Exception as e:
      print('ERROR')
      raise e
      return False

  meta = get_patch(geo, image_ids[0], image_dict['bands'], grid=grid, return_meta=True)
  # final_pixel_values = {k:np.stack([v[name] for name in v.dtype.names], axis=-1) for k,v in pixel_values.items()}
  image_data = {
      'meta': meta,
      'pixel_values': pixel_values
  }

  with open(f'/content/gdrive/MyDrive/{subdir}/{id}.pkl', 'wb') as f:
    pkl.dump(image_data, f)

  return True


In [ ]:
def pull_data_and_write(sites, image_dict, subdir, reproject=False):
  if not os.path.exists(f'/content/gdrive/MyDrive/{subdir}'):
    os.makedirs(f'/content/gdrive/MyDrive/{subdir}')

  print(f'Pulling data from product {image_dict["product"]} for {len(sites)} sites.\n')

  for site in sites:
    print(f'Processing {site["properties"]["SITE_ID"]}...')
    success = get_site_data(site, image_dict, subdir, reproject=reproject)
    if not success:
      print(f'  Exiting after unsuccessful retrieval of {site["properties"]["SITE_ID"]}')
      return
    print('  Success!')

In [ ]:
image_dict = MCD43A2_image_dict()
pull_data_and_write(sites, image_dict, 'modis_a2', reproject=True)

In [ ]:
image_dict = MCD43A4_image_dict()
pull_data_and_write(sites, image_dict, 'modis_a4', reproject=True)